<a href="https://colab.research.google.com/github/shekharkhandelwal1983/Classification/blob/master/keras_tuner_fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install keras-tuner

     |████████████████████████████████| 71kB 7.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78937 sha256=642c1a49c45cb24c04d35d6a333e9402f4bf2b32044a83d5085de39bba97c483
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=3a494492efab9b2b4d83e8de1a27d4e9b51b84bf88b038da34679e95bbb73cc9
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf


In [ ]:
print(tf.__version__)

2.3.0


In [ ]:
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images), 28,28,1)
test_images=test_images.reshape(len(test_images), 28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 17s]
val_accuracy: 0.8840000033378601

Best val_accuracy So Far: 0.9176666736602783
Total elapsed time: 00h 01m 54s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        55360     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                2359360   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 2,416,330
Trainable params: 2,416,330
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1232 - accuracy: 0.9545 - val_loss: 0.2679 - val_accuracy: 0.9108
Epoch 5/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0864 - accuracy: 0.9678 - val_loss: 0.3111 - val_accuracy: 0.9145
Epoch 6/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0618 - accuracy: 0.9775 - val_loss: 0.3368 - val_accuracy: 0.9187
Epoch 7/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0421 - accuracy: 0.9849 - val_loss: 0.4015 - val_accuracy: 0.9132
Epoch 8/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0318 - accuracy: 0.9889 - val_loss: 0.4395 - val_accuracy: 0.9168
Epoch 9/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0266 - accuracy: 0.9913 - val_loss: 0.5049 - val_accuracy: 0.9113
Epoch 10/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0226 - accuracy: 0.9922 - val_loss: 0.5510 - val_accuracy